In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry import Point, LineString, MultiLineString, Polygon, MultiPolygon
from tqdm.notebook import tqdm
import time
from datetime import datetime

import momepy as mm
import networkx as nx

import math

from contextlib import redirect_stdout

import io
import sys

from shapely.ops import nearest_points
from shapely.ops import unary_union

In [ ]:
# проверка остановки в буффере - если даже по вохдуху не попадает - не искать

In [2]:
graph = gpd.read_file('../lines_to_graph/with_ped/new_graph_with_ped.json',
                        encoding='utf-8')

In [3]:
nodes = gpd.read_file('../lines_to_graph/with_ped/all_nodes_with_ped.json',
                        encoding='utf-8')

In [10]:
buildings = gpd.read_file('./all_bui.json',encoding='utf-8')

In [4]:
stps_big = pd.read_csv('../lines_to_graph/total/df_stop_seq_all_n_samobl.csv',
                        encoding='utf-8', sep=';')

In [8]:
stps_big = stps_big[['stop_id', 'stop_name', 'lat', 'lon']]
stps_big = stps_big.drop_duplicates(subset=['stop_id']).reset_index(drop=True)

In [ ]:
lst_geo=[]
lst_lat=list(stps_big.lat)
lst_lon=list(stps_big.lon)
i=0
for i in tqdm(range(len(stps_big))):
    one_pt = Point(lst_lon[i], lst_lat[i])
    lst_geo.append(one_pt)
# 

In [ ]:
gdf_stops = gpd.GeoDataFrame(stps_big, geometry=lst_geo)
gdf_stops.crs='epsg:4326'

In [34]:
# убрать жд пути из графа
graph_ped = graph[graph.type_ped != 'no_ped'].reset_index(drop=True)[['link_id', 'geometry']]

In [13]:
shrt_bui = buildings[['osm_id', 'geometry']].copy()

In [ ]:
def intersect_using_spatial_index(source_gdf, intersecting_gdf):
    """
    Conduct spatial intersection using spatial index for candidates GeoDataFrame to make queries faster.
    Note, with this function, you can have multiple Polygons in the 'intersecting_gdf' and it will return all the points
    intersect with ANY of those geometries.
    """
    source_sindex = source_gdf.sindex
    possible_matches_index = []

    # 'itertuples()' function is a faster version of 'iterrows()'
    for other in intersecting_gdf.itertuples():
        bounds = other.geometry.bounds
        c = list(source_sindex.intersection(bounds))
        possible_matches_index += c

    # Get unique candidates
    unique_candidate_matches = list(set(possible_matches_index))
    possible_matches = source_gdf.iloc[unique_candidate_matches]

    # Conduct the actual intersect
    result = possible_matches.loc[possible_matches.intersects(intersecting_gdf.unary_union)]
    return result
########

In [15]:
np_shbu = shrt_bui.to_numpy()
ind_oi_bui = list(shrt_bui.columns).index('osm_id')
ind_geo_bui = list(shrt_bui.columns).index('geometry')

In [16]:
i=0
one_bui_oi = np_shbu[i, ind_oi_bui]
one_bui_geo = np_shbu[i, ind_geo_bui]

In [60]:
gdf_one_bui = gpd.GeoDataFrame(geometry=[one_bui_geo])
gdf_one_bui.crs = 'epsg:4326'

gdf_buff = gdf_one_bui.copy()
gdf_buff.geometry = gdf_buff.geometry.to_crs('epsg:32637').buffer(1000).to_crs('epsg:4326')

In [36]:
%time gdf_lines_tmp = intersect_using_spatial_index(graph_ped, gdf_poly[['geometry']])

Wall time: 20.3 s


In [37]:
# %time gdf_lines_tmp2 = gpd.sjoin(graph_ped, gdf_poly[['geometry']], how='inner', op='intersects')
# gdf_lines_tmp2 = gdf_lines_tmp2.drop("index_right", axis=1).reset_index(drop=True)

Wall time: 21.9 s


In [47]:
gdf_bui = gpd.GeoDataFrame(geometry=[one_bui_geo])
gdf_bui.crs='epsg:4326'

In [57]:
%time small2 = mm.get_network_id(gdf_bui, gdf_lines_tmp, 'link_id')

Generating centroids...
Generating rtree...


Snapping: 100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 17.86it/s]

Wall time: 260 ms


In [143]:
def find_closest_stop(gdf_one_bui, gdf_stops, gdf_buff):
    uu_stops = gdf_stops.geometry.unary_union
    centr_bui = gdf_one_bui.geometry[0].centroid
    nrst_pt = nearest_points(centr_bui, uu_stops)[1]
    
    nearest_stop_id = list(gdf_stops[gdf_stops.geometry == nrst_pt]['stop_id'])[0]
    nearest_stop_geo = list(gdf_stops[gdf_stops.geometry == nrst_pt]['geometry'])[0]
    if gdf_buff.geometry[0].intersects(nearest_stop_geo):
        return [nearest_stop_id, nearest_stop_geo]
    else:
        return []

In [144]:
%time nearest_stop = find_closest_stop(gdf_one_bui, gdf_stops, gdf_buff)

Wall time: 445 ms


In [145]:
nearest_stop

['stop__10001048', <shapely.geometry.point.Point at 0x27eaad51688>]